In [ ]:
# %pip install torch torchvision --index-url https://download.pytorch.org/whl/cu124

In [ ]:
# %pip install flash_attn-2.7.4%2Bcu124torch2.6.0cxx11abiFALSE-cp310-cp310-win_amd64.whl

In [ ]:
# %pip install "https://github.com/bitsandbytes-foundation/bitsandbytes/releases/download/continuous-release_main/bitsandbytes-1.33.7.preview-py3-none-win_amd64.whl"

In [5]:
# %pip install tqdm

In [ ]:
# %pip install huggingface_hub transformers==4.46.3 tokenizers==0.20.3 einops addict easydict accelerate>=0.26.0

In [1]:
import torch
from flash_attn import flash_attn_func
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
import os

In [2]:
print("flash-attn version:", __import__('flash_attn').__version__)

q = torch.randn(1, 8, 128, 64, device='cuda', dtype=torch.float16)
k = torch.randn(1, 8, 128, 64, device='cuda', dtype=torch.float16)
v = torch.randn(1, 8, 128, 64, device='cuda', dtype=torch.float16)

out = flash_attn_func(q, k, v)
print("Output shape:", out.shape)  # Should be [1, 8, 128, 64]

flash-attn version: 2.7.4
Output shape: torch.Size([1, 8, 128, 64])


In [3]:
model_name = "Jalea96/DeepSeek-OCR-bnb-4bit-NF4"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModel.from_pretrained(
    model_name,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    ),
    device_map="cuda:0",
    trust_remote_code=True,
    use_safetensors=True,
).eval()

print("4-bit model ready for inference!")

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
c:\Users\User\.conda\envs\flashAttn2\lib\site-packages\transformers\quantizers\auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

c:\Users\User\.conda\envs\flashAttn2\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--Jalea96--DeepSeek-OCR-bnb-4bit-NF4. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

4-bit model ready for inference!


In [4]:
prompt = "<image>\n<|grounding|>Convert the document to markdown. "
image_file = '../../tasks/github-readme.png'
output_path = './'

In [5]:
res = model.infer(tokenizer, 
                  prompt=prompt, 
                  image_file=image_file, 
                  output_path = output_path, 
                  base_size = 1024, 
                  image_size = 640, 
                  crop_mode=True, 
                  save_results = True, 
                  test_compress = True)


c:\Users\User\.conda\envs\flashAttn2\lib\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape(

BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([6, 100, 1280])
<|ref|>sub_title<|/ref|><|det|>[[48, 22, 260, 64]]<|/det|>
## TinyBot  

<|ref|>text<|/ref|><|det|>[[44, 95, 962, 236]]<|/det|>
Built to decide smart home actions intelligently, without explicit hard coding. TinyBot is a minimal, text- based conversational model designed to mimic basic understanding of home environment and your needs. It is similar to a chatbot but with an ultra- light neural core. Currently under development.  

<|ref|>sub_title<|/ref|><|det|>[[48, 306, 222, 333]]<|/det|>
## Features  

<|ref|>text<|/ref|><|det|>[[66, 364, 848, 557]]<|/det|>
- Understands simple conversational commands- Controls virtual smart-home devices- Friendly, human-like responses- Built using TensorFlow- Fits within microcontroller memory constraints (few MBs)- Action generator to be added soon  

<|ref|>sub_title<|/ref|><|det|>[[48, 624, 565, 657]]<|/det|>
## Supported Smart Devices  

<|ref|>text<|/ref|><|det|>[[44, 685, 9

image: 0it [00:00, ?it/s]
other: 100%|██████████| 7/7 [00:00<?, ?it/s]
